# Etapa 03 – Modelagem Preditiva com Ajuste para Desbalanceamento

Nesta etapa, realizamos a modelagem preditiva para identificar padrões de desligamento. Após a análise dos primeiros modelos, percebemos um comportamento típico de **desbalanceamento de classes**, onde os modelos previam quase todos os casos como “Não desligamento”, obtendo recall extremamente baixo para a classe 1 (desligamento).

## 🎯 Problema Identificado
- A classe "Desligamento = Sim" corresponde a cerca de 20% dos dados.
- Modelos como Regressão Logística, Random Forest e LightGBM com parâmetros padrão apresentaram **acurácia ilusória** e **recall ≈ 0 para a classe 1**.

## ✅ Soluções aplicadas
1. **Reamostragem com SMOTE**: aumentamos os casos da minoria com técnica de oversampling.
2. **Ajuste de pesos nos classificadores**:
   - `class_weight='balanced'` para LogisticRegression e RandomForest.
   - `is_unbalance=True` para LightGBM.
3. **Avaliação com métricas robustas**:
   - Matriz de confusão
   - ROC AUC Score
   - F1-score da classe minoritária
4. **Ajuste dinâmico de threshold**:
   - Avaliação de thresholds entre 0.1 e 0.9
   - Verificação do melhor ponto de equilíbrio entre precisão e recall
   - LightGBM com threshold 0.2 apresentou o melhor F1-score (0.39)

Essas medidas foram implementadas para melhorar a capacidade preditiva sobre os casos de desligamento real e evitar a ilusão de desempenho pela acurácia geral.

## Modelos aplicados:
- Regressão Logística
- Random Forest
- LightGBM

Os resultados foram comparados com as abordagens padrão, com técnicas de balanceamento e ajuste de threshold.

O script atual aplica todas essas estratégias de forma integrada, possibilitando testes e comparações para escolha do melhor modelo com base nas métricas de negócio.


In [9]:
# Modelagem com ajuste para desbalanceamento e thresholds dinâmicos
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.metrics import (
    precision_score, recall_score, f1_score, roc_auc_score
)
from sklearn.model_selection import train_test_split
import sys
import os
sys.path.append(os.path.abspath("../"))
from scripts.func import preparar_dados_rh

# Preparar base com novas features e pré-processamento
df_encoded, X_train, X_test, y_train, y_test = preparar_dados_rh(qtd_amostras=100_000)

# Modelos com ajustes de balanceamento
models = {
    'Regressao Logistica': LogisticRegression(max_iter=1000, class_weight='balanced'),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'),
    'LightGBM': lgb.LGBMClassifier(random_state=42, is_unbalance=True)
}

# Avaliação por thresholds
thresholds = np.arange(0.1, 0.91, 0.1)

# Avaliação com ajuste de threshold
for nome, modelo in models.items():
    print(f"\n📌 Modelo: {nome}")
    modelo.fit(X_train, y_train)
    y_proba = modelo.predict_proba(X_test)[:, 1]

    print("Threshold | Precision | Recall | F1-Score | ROC AUC")
    print("-" * 55)

    for t in thresholds:
        y_pred_thresh = (y_proba >= t).astype(int)
        precision = precision_score(y_test, y_pred_thresh, zero_division=0)
        recall = recall_score(y_test, y_pred_thresh, zero_division=0)
        f1 = f1_score(y_test, y_pred_thresh, zero_division=0)
        roc_auc = roc_auc_score(y_test, y_proba)

        print(f"{t:9.2f} | {precision:9.2f} | {recall:6.2f} | {f1:8.2f} | {roc_auc:.4f}")



📌 Modelo: Regressao Logistica
Threshold | Precision | Recall | F1-Score | ROC AUC
-------------------------------------------------------
     0.10 |      0.21 |   1.00 |     0.34 | 0.6137
     0.20 |      0.21 |   1.00 |     0.34 | 0.6137
     0.30 |      0.21 |   0.99 |     0.35 | 0.6137
     0.40 |      0.23 |   0.88 |     0.36 | 0.6137
     0.50 |      0.27 |   0.57 |     0.36 | 0.6137
     0.60 |      0.32 |   0.22 |     0.26 | 0.6137
     0.70 |      0.35 |   0.03 |     0.05 | 0.6137
     0.80 |      0.67 |   0.00 |     0.00 | 0.6137
     0.90 |      0.00 |   0.00 |     0.00 | 0.6137

📌 Modelo: Random Forest
Threshold | Precision | Recall | F1-Score | ROC AUC
-------------------------------------------------------
     0.10 |      0.22 |   0.95 |     0.35 | 0.6075
     0.20 |      0.26 |   0.62 |     0.36 | 0.6075
     0.30 |      0.31 |   0.20 |     0.24 | 0.6075
     0.40 |      0.35 |   0.04 |     0.08 | 0.6075
     0.50 |      0.44 |   0.00 |     0.01 | 0.6075
     0.60 |   